### FabienDaniel's kernel (https://www.kaggle.com/fabiendaniel/detecting-malwares-with-lgbm/comments#440611) has been my reference throughout this competition. 
#### His categorization of the features is very clean and he lists three principal kinds of features::
* Numerical features (which includes categories represented as numbers and true numerical features)
*  Binary features (these could also be considered Categorical, but let's make this distinction and see what we get for now)
* Categorical features

## Here we analyze the* numerical features ('true' and 'false')* while the categorical features (binary and otherwise) will be analyzed in another kernel

### In this kernel I plan to study these kinds of features in some detail, and analyze which ones are more important to the AUC, so that we have a better idea of what gets more importance as we go into the deeper end of the competition

### *Note:*: We will generally not put much effort into getting high accuracies with this kernel. The goal is to understand the relative importances of the various kinds of features to the target. Once we have that knowledge in place, then we can hopefully get a better idea of what kind of feature engineering will actually be of use. 

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb


import warnings
warnings.filterwarnings('ignore')

import gc
import time
import sys
import datetime

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn import preprocessing as pp
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer

from sklearn import metrics
# Plotly library
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)


from tqdm import tqdm

#### Using Theo Viel's method (https://www.kaggle.com/theoviel/load-the-totality-of-the-data)

In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

### Let's read in a fraction of the training set

In [ ]:
%%time
nrows = 1000000
#_______________________________________________________________________________
# retained_columns = numerical_columns + categorical_columns
train = pd.read_csv('../input/train.csv',
                    nrows = nrows,
#                     usecols = retained_columns,
                    dtype = dtypes)

In [ ]:
train.head()

# 1. Numerical Features
## i. "True" Numerical Features
### First let's look at the numerical columns

In [ ]:
target = train['HasDetections']
train.drop('HasDetections', inplace=True, axis=1)

In [ ]:
num_datatypes = ['int8', 'int16', 'int32', 'float16', 'float32']
numerical_columns = [c for c,v in dtypes.items() if v in num_datatypes]

### Of these, only a handful of columns (as Fabien points out) are really numerical

In [ ]:
true_numerical_columns = [
    'Census_ProcessorCoreCount',
    'Census_PrimaryDiskTotalCapacity',
    'Census_SystemVolumeTotalCapacity',
    'Census_TotalPhysicalRAM',
    'Census_InternalPrimaryDiagonalDisplaySizeInInches',
    'Census_InternalPrimaryDisplayResolutionHorizontal',
    'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_InternalBatteryNumberOfCharges'
]

In [ ]:
false_numerical_columns = [col for col in numerical_columns if col not in true_numerical_columns]

### Let's look at how accurate our calculations would be just for the true_numerical features

In [ ]:
train_true_num = train[true_numerical_columns]
train_true_num.head(2)

### Let's try an AUC calculation just based on these features

In [ ]:
train_true_num['MachineIdentifier'] = train['MachineIdentifier']

In [ ]:
lgb_params = {'num_leaves': 60,
         'min_data_in_leaf': 60, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.1,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 42,
         "verbosity": -1}

In [ ]:
train_true_num.head()

In [ ]:
train_true_num.shape

In [ ]:
gc.collect()

In [ ]:
folds = KFold(n_splits=3, shuffle=True, random_state=42)
oof = np.zeros(len(train_true_num))
# categorical_columns = [c for c in categorical_columns if c not in ['MachineIdentifier']]
features = [c for c in train_true_num.columns if c not in ['MachineIdentifier']]
# predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_true_num.values, target.values)):
    print("Fold No.{}".format(fold_+1))
    trn_data = lgb.Dataset(train_true_num.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
#                            categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(train_true_num.iloc[val_idx][features],
                           label=target.iloc[val_idx],
#                            categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(lgb_params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train_true_num.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print("Time elapsed: {:<5.2}s".format((time.time() - start_time) / 3600))
    score[fold_] = metrics.roc_auc_score(target.iloc[val_idx], oof[val_idx])

feat_num_true_score = metrics.roc_auc_score(target, oof)
print("CV score: {:<8.5f}".format(feat_num_true_score))
print("Total time elapsed: {:<5.2}s".format(time.time() - start))

### Now let's plot the feature importances

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(8,8))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds) -- True Numerical Features')
plt.tight_layout()

### Let's add some polynomial features to this data frame of true numerical features

### The code below is from https://stackoverflow.com/questions/36728287/sklearn-preprocessing-polynomialfeatures-how-to-keep-column-names-headers-of

In [ ]:
def PolynomialFeatures_labeled(input_df,power):
    '''Basically this is a cover for the sklearn preprocessing function. 
    The problem with that function is if you give it a labeled dataframe, it ouputs an unlabeled dataframe with potentially
    a whole bunch of unlabeled columns. 

    Inputs:
    input_df = Your labeled pandas dataframe (list of x's not raised to any power) 
    power = what order polynomial you want variables up to. (use the same power as you want entered into pp.PolynomialFeatures(power) directly)

    Ouput:
    Output: This function relies on the powers_ matrix which is one of the preprocessing function's outputs to create logical labels and 
    outputs a labeled pandas dataframe   
    '''
    poly = pp.PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + " x " + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df

### As a basic step, let's set all the missing values to zero (will this skew the distribution?)

In [ ]:
train_true_num_copy = train_true_num.drop('MachineIdentifier', axis=1)

In [ ]:
train_true_num_pow2 = PolynomialFeatures_labeled(train_true_num_copy.fillna(0), 2)
train_true_num_pow2.head()

In [ ]:
train_true_num_pow2['MachineIdentifier'] = train_true_num['MachineIdentifier']
train_true_num_pow2.shape

In [ ]:
train_true_num_pow2 = reduce_mem_usage(train_true_num_pow2)

In [ ]:
gc.collect()

### Let's look at the CV score and the feature importances of these

In [ ]:
folds = KFold(n_splits=3, shuffle=True, random_state=42)
oof = np.zeros(len(train_true_num_pow2))
# categorical_columns = [c for c in categorical_columns if c not in ['MachineIdentifier']]
features = [c for c in train_true_num_pow2.columns if c not in ['MachineIdentifier']]
# predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_true_num_pow2.values, target.values)):
    print("Fold No.{}".format(fold_+1))
    trn_data = lgb.Dataset(train_true_num_pow2.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
#                            categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(train_true_num_pow2.iloc[val_idx][features],
                           label=target.iloc[val_idx],
#                            categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(lgb_params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train_true_num_pow2.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print("Time elapsed: {:<5.2}s".format((time.time() - start_time) / 3600))
    score[fold_] = metrics.roc_auc_score(target.iloc[val_idx], oof[val_idx])

feat_num_pow2_score = metrics.roc_auc_score(target, oof)
print("CV score: {:<8.5f}".format(feat_num_pow2_score))
print("Total time elapsed: {:<5.2}s".format(time.time() - start))

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(12,12))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds) -- True Numerical Features (including Polynomials)')
plt.tight_layout()

### The CV accuracy is marginally lower than before, but in the Feature Importances we do see a couple of things that might turn out to be important. Census_TotalPhysicalRAM gets pushed down (as compared to when we just used the true numerical features without polynomials), and the product of Census_TotalPhysicalRam and Census_InternalPrimaryDiagonalDisplaySizeInInches becomes very important. 

### Must check how much of this is due to the relatively arbitrary handling of nans

## ii. "False" Numerical Features
### Let's look at the other 'false' numerical features

### Let's check the AUC and feature_importances for these

In [ ]:
del false_numerical_columns[-1]

In [ ]:
train_false_num = train[false_numerical_columns]
train_false_num['MachineIdentifier'] = train['MachineIdentifier']
train_false_num.head(2)

In [ ]:
train_false_num.shape

In [ ]:
train_false_num = reduce_mem_usage(train_false_num)

In [ ]:
gc.collect()

In [ ]:
folds = KFold(n_splits=3, shuffle=True, random_state=42)
oof = np.zeros(len(train_false_num))
# categorical_columns = [c for c in categorical_columns if c not in ['MachineIdentifier']]
features = [c for c in train_false_num.columns if c not in ['MachineIdentifier']]
# predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()
start_time= time.time()
score = [0 for _ in range(folds.n_splits)]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_false_num.values, target.values)):
    print("Fold No.{}".format(fold_+1))
    trn_data = lgb.Dataset(train_false_num.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
#                            categorical_feature = categorical_columns
                          )
    val_data = lgb.Dataset(train_false_num.iloc[val_idx][features],
                           label=target.iloc[val_idx],
#                            categorical_feature = categorical_columns
                          )

    num_round = 10000
    clf = lgb.train(lgb_params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(train_false_num.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print("Time elapsed: {:<5.2}s".format((time.time() - start_time) / 3600))
    score[fold_] = metrics.roc_auc_score(target.iloc[val_idx], oof[val_idx])

feat_num_false_score = metrics.roc_auc_score(target, oof)
print("CV score: {:<8.5f}".format(feat_num_false_score))
print("Total time elapsed: {:<5.2}s".format(time.time() - start))

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(12,12))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds) -- False Numerical Features')
plt.tight_layout()

### So a considerably higher CV for using just the 'false' numeric features

### In summary, we analyze the numerical features in the Microsoft Malware Detection training set. We find that just utilizing the 'true' numerical features gives us a marginally better CV than utilizing some polynomial features of the same (in addition). However, we do see certain combinations of features contributing large feature importances, which we would do well to keep in mind for competition entries. 
### Just using the 'false' numerical features gives us a much higher AUC score, and we might want to test out polynomial features for features like 'AvProductStatesIdentifier', 'AvProductsInstalled' and 'Census_ProcessorModelIdentifier'. 

### Most of the 'false' numeric features can be probably represented as some variety of categorical features so this is also something to probably look into. 

